In [4]:
import json
from itertools import product

# Data structure to organize pair conditions
cam_velocities_per_motion_condition_given_obj_vel = {
    "contrast": {
        30.0: 0,
        8.0: 0,
        15.0: 0,
    }
}

object_velocities = [
    15.0, 
    # 30.0
    ]

object_directions = {
    "horizontal_left": (-1, 0),
    "horizontal_right": (1, 0)
}

initial_ball_placements = {
    "horizontal_left": {
        30.0: (0.57, 0.5),
        8.0: (0.52, 0.5),
        15.0: (0.53, 0.5),
    },
    "horizontal_right": {
        30.0: (0.43, 0.5),
        8.0: (0.48, 0.5),
        15.0: (0.47, 0.5),
    }
}

initial_crosshair_placements = {
    "horizontal_left": {
        30.0: (0.5, 0.5),
        8.0: (0.5, 0.5),
        15.0: (0.5, 0.5),
    },
    "horizontal_right": {
        30.0: (0.5, 0.5),
        8.0: (0.5, 0.5),
        15.0: (0.5, 0.5),
    }
}

duration_mapping = {
    "horizontal_left": {
        30.0: 2,
        8.0: 2,
        15.0: 2,
    },
    "horizontal_right": {
        30.0: 2,
        8.0: 2,
        15.0: 2,
    }
}

contrast_thresholds = [1.01, 1.02, 1.04, 1.08]

ramp_location = (-1, 0)

# Helper function to generate trial segments
def generate_segments_2_speeds(object_direction, object_velocity, duration, velocity_delta):
    return [
        {
            "object_motion": object_direction,
            "object_speed": object_velocity,
            "camera_motion": (0, 0),
            "camera_speed": 0,
            "duration": duration
        },
        {
            "object_motion": object_direction,
            "object_speed": object_velocity-velocity_delta,
            "camera_motion": (0, 0),
            "camera_speed": 0,
            "duration": duration
        },
        {
            "object_motion": object_direction,
            "object_speed": object_velocity+velocity_delta,
            "camera_motion": (0, 0),
            "camera_speed": 0,
            "duration": duration
        }
    ]

# Generate trial data
unique_trials = []
trial_id = 0

for obj_velocity, (obj_motion_type, obj_direction) in product(
        object_velocities,
        object_directions.items()):
    
    for cam_condition, cam_velocity in cam_velocities_per_motion_condition_given_obj_vel.items():
        for threshold in contrast_thresholds:
            
            initial_ball_placement = initial_ball_placements[obj_motion_type][obj_velocity]
            initial_crosshair_placement = initial_crosshair_placements[obj_motion_type][obj_velocity]
            
            duration = duration_mapping[obj_motion_type][obj_velocity]
            
            segments = generate_segments_2_speeds(obj_direction, obj_velocity, duration)

            trial = {
                "trial_id": trial_id,
                "initial_ball_placement": {"x": initial_ball_placement[0], "y": initial_ball_placement[1]},
                "initial_crosshair_placement": {"x": initial_crosshair_placement[0], "y": initial_crosshair_placement[1]},
                "segments": segments
            }
            
            unique_trials.append(trial)
            trial_id += 1

# Write trials to JSON
output_file = "trial_ids_segments/trials.json"
with open(output_file, mode='w') as file:
    json.dump(unique_trials, file, indent=4)

print(f"Trials saved to {output_file}")
print(len(unique_trials))


Trials saved to trial_ids_segments/trials.json
8
